This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

Step 1: Setup
Simply install the provided requirements.txt file by running the following command:
```bash
pip install -r requirements.txt
```

Step 2 & 3: Generating Fake Listings Using LLM and Saving Them

In [110]:
import openai
import os

from langchain.prompts import FewShotPromptTemplate, PromptTemplate

from langchain.llms import OpenAI


In [111]:
from langchain.prompts import PromptTemplate

listing_template = PromptTemplate(template=
"""Neighborhood: {neighborhood}
Price: ${price}
Bedrooms: {bathrooms}
Bathrooms: {bedrooms}
House Size: {house_size} sqft

Description: {description}

Neighborhood Description: {neighborhood_description}
""",
                                  input_variables=["neighborhood", "price", "bedrooms", "bathrooms",
                                                   "house_size", "description", "neighborhood_description"]
                                  )



In [112]:
example_inputs = [
    {
        "neighborhood": "Downtown Abbey",
        "price": 300000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A beautiful appartment with a large terrace overlooking the city's skyscrapers with 3 well-lit bedrooms and 2 bathrooms means you have enough room even for the occasional guest coming over to celebrate new year's with you :)",
        "neighborhood_description": "A vibrant neighborhood which is very centrally located and specially well connected public transport connections means you are close to everything you need for your everyday life from groceries to cafes and restaurants and shopping."
    },
    {
        "neighborhood": "City Heights",
        "price": 500000,
        "bedrooms": 4,
        "bathrooms": 3,
        "house_size": 3000,
        "description": "A cozy house with a wonderful garden overlooking the city from its hillside location",
        "neighborhood_description": "Located in the city outskirts, this neighborhood is known for its great views and quiet streets with a very family-friendly atmosphere. In no time you are in the midst of a beautiful forest where you can calm all your senses and breath in the cool fresh air. Sightings of deer and other wildlife are common."
    },
    {
        "neighborhood": "Green Oaks",
        "price": 800000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A luxurious villa with a large garden and a swimming pool with a winter garden, 2 parking spots, and a garage as well as a large terrace and wonderful finishing.",
        "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
    }
]

In [113]:
from langchain.prompts import FewShotPromptTemplate

few_shot_template = FewShotPromptTemplate(
    example_prompt=listing_template,
    input_variables=["input"],
    examples=example_inputs,
    suffix="Use the examples above to generate the following: {input}",
)

listing_seperator = "===="
instruction = "Generate {} listings of houses. Be creative in regards neighborhood_description, and description and keep it confined to three sentences each. Keep the prices under $1500000, the number of bedrooms under 6 and the number of bathrooms under 3 and the total area under 3500sqft. VERY IMPORTANT: Output the results in the same format as the examples keeping even the order of properties the same as in the examples. Add the characters '%s' before each new listing" % listing_seperator
num_listings = 10
prompt_to_use = few_shot_template.format(input=instruction.format(num_listings))



In [114]:
print(prompt_to_use)

Neighborhood: Downtown Abbey
Price: $300000
Bedrooms: 2
Bathrooms: 3
House Size: 2000 sqft

Description: A beautiful appartment with a large terrace overlooking the city's skyscrapers with 3 well-lit bedrooms and 2 bathrooms means you have enough room even for the occasional guest coming over to celebrate new year's with you :)

Neighborhood Description: A vibrant neighborhood which is very centrally located and specially well connected public transport connections means you are close to everything you need for your everyday life from groceries to cafes and restaurants and shopping.


Neighborhood: City Heights
Price: $500000
Bedrooms: 3
Bathrooms: 4
House Size: 3000 sqft

Description: A cozy house with a wonderful garden overlooking the city from its hillside location

Neighborhood Description: Located in the city outskirts, this neighborhood is known for its great views and quiet streets with a very family-friendly atmosphere. In no time you are in the midst of a beautiful forest whe

In [115]:
import os

os.environ["OPENAI_API_KEY"] =
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [116]:
from langchain.chat_models import ChatOpenAI

model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name, temperature=0.0, max_tokens=2000)

/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/tumaaa/work/repos/udacity_nano_degree_generative_ai/.venv/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [117]:
result = llm(prompt=prompt_to_use)

In [118]:
print(f"LLM result:\n{result}")

LLM result:
====
Neighborhood: Sunnyvale Heights
Price: $700000
Bedrooms: 4
Bathrooms: 2
House Size: 2500 sqft

Description: A charming family home with a spacious backyard perfect for hosting summer BBQs and gatherings. The open floor plan and large windows allow for plenty of natural light to brighten up the living spaces.

Neighborhood Description: Sunnyvale Heights is a peaceful neighborhood with tree-lined streets and friendly neighbors. Close to top-rated schools, parks, and shopping centers, this area offers a perfect balance of convenience and tranquility. Enjoy morning walks in the nearby Sunnyvale Park or grab a cup of coffee at the local cafe.


====
Neighborhood: Lakeview Terrace
Price: $900000
Bedrooms: 3
Bathrooms: 2
House Size: 2200 sqft

Description: A modern lakefront home with stunning views of the water and surrounding mountains. The spacious deck is perfect for enjoying morning coffee or evening sunsets with loved ones.

Neighborhood Description: Lakeview Terrace is

In [119]:
import pandas as pd
import locale
import re

# Set the locale to 'en_US.UTF-8' for parsing numbers with commas
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Split the result into individual listings
listings = result.split(listing_seperator)[1:]
# print(f"Here are the listings in a list:\n{listings}")

# Define the columns based on input_variables
columns = ["neighborhood", "price", "bedrooms", "bathrooms", "house_size", "description", "neighborhood_description"]

# Parse the listings into a list of dictionaries
data = []
for listing in listings:
    print(f"Parsing listing: {listing[1:150]} ...\n")
    try:
        entry = {
            "neighborhood": re.search(r"^Neighborhood: (.+)$", listing, re.MULTILINE).group(1),
            "price": locale.atoi(re.search(r"^Price: \$(.+)$", listing, re.MULTILINE).group(1)),
            "bedrooms": int(re.search(r"^Bedrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "bathrooms": int(re.search(r"^Bathrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "house_size": locale.atoi(re.search(r"^House Size: (.+) sqft$", listing, re.MULTILINE).group(1)),
            "description": re.search(r"^Description: (.+)$", listing, re.MULTILINE).group(1),
            "neighborhood_description": re.search(r"^Neighborhood Description: (.+)$", listing, re.MULTILINE).group(1)
        }

        data.append(entry)
    except Exception as e:
        print("Error parsing listing")
        print(listing)
        print(f"Because of \n{e}")
        print("\n")
        continue

# Create a DataFrame
df = pd.DataFrame(data, columns=columns)

Parsing listing: Neighborhood: Sunnyvale Heights
Price: $700000
Bedrooms: 4
Bathrooms: 2
House Size: 2500 sqft

Description: A charming family home with a spacious ba ...

Parsing listing: Neighborhood: Lakeview Terrace
Price: $900000
Bedrooms: 3
Bathrooms: 2
House Size: 2200 sqft

Description: A modern lakefront home with stunning view ...

Parsing listing: Neighborhood: Oakwood Estates
Price: $600000
Bedrooms: 3
Bathrooms: 2
House Size: 1800 sqft

Description: A cozy ranch-style home nestled in a quiet  ...

Parsing listing: Neighborhood: Maple Grove
Price: $400000
Bedrooms: 2
Bathrooms: 1
House Size: 1500 sqft

Description: A charming cottage-style home with a cozy firep ...

Parsing listing: Neighborhood: Pinecrest Heights
Price: $550000
Bedrooms: 3
Bathrooms: 2
House Size: 2000 sqft

Description: A spacious split-level home with a large  ...

Parsing listing: Neighborhood: Willow Creek
Price: $750000
Bedrooms: 4
Bathrooms: 2
House Size: 2300 sqft

Description: A beautiful colonial

Let's view the dataframe and assert the generated input makes sense and finally save into a csv.

In [120]:
df.head(n=num_listings)

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
0,Sunnyvale Heights,700000,4,2,2500,A charming family home with a spacious backyar...,Sunnyvale Heights is a peaceful neighborhood w...
1,Lakeview Terrace,900000,3,2,2200,A modern lakefront home with stunning views of...,Lakeview Terrace is a serene community with ac...
2,Oakwood Estates,600000,3,2,1800,A cozy ranch-style home nestled in a quiet cul...,Oakwood Estates is a family-friendly neighborh...
3,Maple Grove,400000,2,1,1500,A charming cottage-style home with a cozy fire...,Maple Grove is a quaint neighborhood with tree...
4,Pinecrest Heights,550000,3,2,2000,A spacious split-level home with a large backy...,Pinecrest Heights is a quiet and friendly neig...
5,Willow Creek,750000,4,2,2300,A beautiful colonial-style home with a wrap-ar...,Willow Creek is a picturesque neighborhood wit...
6,Cedar Ridge,650000,3,2,1900,A cozy bungalow with a spacious backyard and a...,Cedar Ridge is a quiet and family-friendly nei...
7,Sunset Hills,850000,5,3,3200,A spacious family home with a large backyard a...,Sunset Hills is a peaceful neighborhood with r...
8,Riverfront Estates,950000,4,2,2400,A luxurious waterfront home with a private doc...,Riverfront Estates is an exclusive community w...
9,Meadowbrook Heights,500000,3,2,1800,A charming ranch-style home with a spacious ba...,Meadowbrook Heights is a family-friendly neigh...


In [121]:
df.to_csv("listings.csv", index=False)
